# Descriptive Statistics
### base on Chua, Alton Y.K. and Banerjee, Snehasish 2015
#### http://onlinelibrary.wiley.com/doi/10.1002/asi.23180/abstract

In [2]:
import sys
sys.path.append("../pylinguistics/pylinguistics/")

import Pylinguistics as pl
import pandas as pd
import plotsfunc as pf
import numpy as np
from scipy.stats import spearmanr
%matplotlib inline

#reviews_features = pd.read_csv('reviews.csv.gz', compression='gzip')
reviews_features = pd.read_csv('reviews-amazon.csv')

In [3]:
## adiciona coluna de helpfulness no corpus
def helpf(x): 
    try:
        return float ( float(x['thumbsup']) / (float(x['thumbsup']) + float(x['thumbsdown'])) )
    except:
        return 0
    
def thumbssum(x): 
    try:
        return int(x['thumbsup']) + int(x['thumbsdown']) 
    except:
        return 0
    
def fillnanup(x): 
    try:
        return float(x['thumbsup'])
    except:
        return 0
    
def fillnandown(x): 
    try:
        return float(x['thumbsdown'])
    except:
        return 0
    
reviews_features['thumbsup'] = reviews_features.apply(fillnanup,axis=1)
reviews_features['thumbsdown'] = reviews_features.apply(fillnandown,axis=1)
reviews_features['helpfulness'] = reviews_features.apply(helpf,axis=1)
reviews_features['thumbstotal'] = reviews_features.apply(thumbssum,axis=1)

In [4]:
def mean_data(df):
    mean_stats = []
    mean_stats.append(df.thumbstotal.mean())
    mean_stats.append(df.stars.mean())
    mean_stats.append(df.word_count.mean())
    mean_stats.append(df.helpfulness.mean())
    mean_stats.append(len(df))
    return mean_stats

def std_data(df):
    std_stats = []
    std_stats.append(df.thumbstotal.std())
    std_stats.append(df.stars.std())
    std_stats.append(df.word_count.std())
    std_stats.append(df.helpfulness.std())
    std_stats.append(len(df))
    return std_stats

columns = ['TOT','RAT','DEP','HEL','LEN']

# full dataset
stats = pd.DataFrame(data={'mean_full': np.array(mean_data(reviews_features))}, index=columns)
stats['std_full'] = std_data(reviews_features)

In [5]:
# split low DEP dataset
low_reviews = reviews_features[reviews_features.word_count <= reviews_features.word_count.median()]

In [6]:
# split high DEP dataset
high_reviews = reviews_features[reviews_features.word_count > reviews_features.word_count.median()]

In [7]:
stats['mean_low'] = mean_data(low_reviews)
stats['std_low'] = std_data(low_reviews)
stats['mean_high'] = mean_data(high_reviews)
stats['std_high'] = std_data(high_reviews)

## Descriptive Table Buscape Reviews

In [20]:
stats

,mean_full,std_full,mean_low,std_low,mean_high,std_high
TOT,6.187302,13.557119,4.010325,7.726384,8.407396,17.337848
RAT,3.823951,1.188895,4.096380,1.055879,3.546161,1.251068
DEP,59.670049,57.907309,23.922746,10.535257,96.125353,63.463516
HEL,0.729849,0.377087,0.676410,0.414662,0.784340,0.325555
LEN,32226.000000,32226.000000,16271.000000,16271.000000,15955.000000,15955.000000


## Descriptive Table Amazon Reviews

In [8]:
stats

,mean_full,std_full,mean_low,std_low,mean_high,std_high
TOT,9.741407,32.152690,5.410121,8.017842,14.125733,44.463730
RAT,3.393389,1.578104,3.412961,1.641476,3.373577,1.511041
DEP,117.650124,144.247460,40.880616,16.966970,195.359722,171.971119
HEL,0.403156,0.164908,0.375070,0.187611,0.431586,0.132242
LEN,35001.000000,35001.000000,17607.000000,17607.000000,17394.000000,17394.000000


## Correlations

In [21]:
def corr_table(df):
    # create new dict for correlation
    pearson_helpfull_list = []
    spearmanr_helpfull_list = []
    pearson_up_list = []
    spearmanr_up_list = []
    pearson_down_list = []
    spearmanr_down_list = []
    
    # build columns list
    df_columns = df.select_dtypes(include=['float64','int']).columns.drop('helpfulness').drop('thumbsup').drop('thumbsdown')
    
    #compute correlation
    for i in df_columns:
        pearson_helpfull_list.append(df.helpfulness.corr(df[i]))
        spearmanr_helpfull_list.append(spearmanr(df.helpfulness, df[i])[0])
        pearson_up_list.append(df.thumbsup.corr(df[i]))
        spearmanr_up_list.append(spearmanr(df.thumbsup, df[i])[0])
        pearson_down_list.append(df.thumbsdown.corr(df[i]))
        spearmanr_down_list.append(spearmanr(df.thumbsdown, df[i])[0])

    corr_df = pd.DataFrame(data={'pearson_helpfull': np.array(pearson_helpfull_list)}, index=df_columns)
    corr_df['spearmanr_helpfull'] = spearmanr_helpfull_list
    corr_df['pearson_up'] = pearson_up_list
    corr_df['spearmanr_up'] = spearmanr_up_list
    corr_df['pearson_down'] = pearson_down_list
    corr_df['spearmanr_down'] = spearmanr_down_list
    return corr_df

### Top 10 full dataset correlation

In [22]:
corr_table(reviews_features).sort_values('pearson_helpfull',ascending=False).head(10)

,pearson_helpfull,spearmanr_helpfull,pearson_up,spearmanr_up,pearson_down,spearmanr_down
sentence_count,0.141733,NaN,0.206188,NaN,0.046840,NaN
syllable_count,0.134348,NaN,0.218057,NaN,0.053729,NaN
word_count,0.126666,NaN,0.218488,NaN,0.057790,NaN
functionalIncidence,0.067938,NaN,0.072279,NaN,0.016081,NaN
percentile_90_sentence_length,0.050461,NaN,0.097291,NaN,0.031930,NaN
ConnectiveTemporalIncidence,0.030754,NaN,0.013497,NaN,-0.004567,NaN
Unnamed: 0,0.030495,NaN,-0.023120,NaN,-0.048140,NaN
percentile_90_word_length,0.029200,NaN,0.008188,NaN,-0.006464,NaN
percentile_75_word_length,0.027247,NaN,0.010063,NaN,-0.010924,NaN
percentile_75_sentence_length,0.026271,NaN,0.065270,NaN,0.025491,NaN


### Top 10 low DEP dataset correlation

In [23]:
corr_table(low_reviews).sort_values('pearson_helpfull',ascending=False).head(10)

,pearson_helpfull,spearmanr_helpfull,pearson_up,spearmanr_up,pearson_down,spearmanr_down
syllable_count,0.141626,NaN,0.147265,NaN,-0.002371,NaN
word_count,0.135052,NaN,0.155602,NaN,0.006513,NaN
sentence_count,0.118570,NaN,0.104107,NaN,-0.022372,NaN
adjectiveIncidence,0.070917,NaN,-0.002002,NaN,-0.043061,NaN
stars,0.053487,NaN,-0.051734,NaN,-0.055837,NaN
percentile_90_sentence_length,0.047815,NaN,0.073190,NaN,0.018485,NaN
Unnamed: 0,0.044402,NaN,-0.011098,NaN,-0.042966,NaN
ConnectiveTemporalIncidence,0.033895,NaN,0.004598,NaN,-0.017077,NaN
percentile_75_sentence_length,0.031798,NaN,0.059959,NaN,0.021609,NaN
ConnectiveCasualIncidence,0.027588,NaN,0.003578,NaN,-0.015618,NaN


### Top 10 high DEP dataset correlation

In [24]:
corr_table(high_reviews).sort_values('pearson_helpfull',ascending=False).head(10)

,pearson_helpfull,spearmanr_helpfull,pearson_up,spearmanr_up,pearson_down,spearmanr_down
sentence_count,0.089830,0.050144,0.133964,0.174297,0.046498,0.015717
ContentDiversty,0.084987,0.055985,-0.017535,-0.047876,-0.044519,-0.068071
avg_syllables_per_word,0.078178,0.059042,0.001261,0.029503,-0.031350,-0.046681
syllable_count,0.063939,0.022452,0.132193,0.183437,0.047304,0.040973
LexicalDiversty,0.062494,0.047847,-0.049236,-0.080315,-0.053996,-0.071041
percentile_90_word_length,0.058238,0.042701,0.013736,0.043441,-0.025603,-0.025877
word_count,0.050844,0.009013,0.132179,0.179819,0.052581,0.052764
percentile_75_word_length,0.049325,0.041940,0.014359,0.025177,-0.015301,-0.031352
adjectiveIncidence,0.048812,0.057356,-0.009556,0.025523,-0.028984,-0.046346
stars,0.045630,0.062980,-0.051950,-0.073512,-0.056922,-0.081744
